# Rankings
#### Aquesta setmana hem aconseguit executar l'script de rankings en el jupyter, per aconseguir-ho hem fet:
####     -Crear el notebook al jupyter.
####     -Guardar.
####     -Obrir amb el Canopy, llavors s'obrirà el notebook al jupyter.
####     -Executar (perque fent aquests pasos utilizta les llibreries del Canopy).

#### Comencem fent els import basics, incluint el params (inclou la direcció dels arxius).

In [1]:
import numpy as np
import time
#import sys
# Add the root path (the path above this one) to the pythonpath.
# sys.path.insert(0,../)
from params import get_params

params = get_params()

print ("Number of clusters:", params['descriptor_size'])
print ("Descriptor type:",params['descriptor_type'])
print ("Keypoint detector:", params['keypoint_type'])
print ("Resize dimension:", params['max_size'])
print ("Distance metric:", params['distance_type'])

('Number of clusters:', 1024)
('Descriptor type:', 'SIFT')
('Keypoint detector:', 'SIFT')
('Resize dimension:', 300)
('Distance metric:', 'euclidean')


## Feature extraction with Bag of Words

#### Utilitzem la funció get_features per calcular els descriptors locals de totes les imatges d'entrenament.

In [2]:
import get_features as GF
# Make sure that we are using training images only !
params['split'] = 'train'

t = time.time()
X, pca, scaler = GF.stack_features(params)

print ("Done. Time elapsed:", time.time() - t)
print (np.shape(X))

('Done. Time elapsed:', 127.92300009727478)
(216506L, 128L)


#### Despres d'acabar amb l'entrenament de features podem començar a entrenar el nostre codebook. Utilitzem MiniBatchKMeans perque es molt mes ràpid que el KMeans normal.

In [3]:
t = time.time()
GF.train_codebook(params,X)

print ("Done. Time elapsed:", time.time() - t)

('Done. Time elapsed:', 27.914000034332275)


#### Una vegada tenim el codebook, podem fer les asignacions a cada imatge i fer el Bag of Words.

In [4]:
t = time.time()
GF.get_features(params)

print ("Done. Time elapsed for training set:", time.time() - t)

('Done. Time elapsed for training set:', 120.07500004768372)


In [5]:
# Switch to validation set
params['split'] = 'val'

t = time.time()
# Run again
GF.get_features(params)

print ("Done. Time elapsed for validation set:", time.time() - t)

('Done. Time elapsed for validation set:', 46.930999994277954)


## Generem el ranking

In [6]:
from rank import *

t = time.time()
rank(params)

print ("Done. Time elapsed:", time.time() - t)

('Done. Time elapsed:', 1.194000005722046)


## Avaluació

In [7]:
import eval_rankings as ER

ap_list, dict_ = ER.eval_rankings(params)

print ("Number of queries:", len(ap_list))
print ("Mean Average Precision", np.mean(ap_list)),

('Number of queries:', 120)
('Mean Average Precision', 0.24347167699973812)


In [8]:
for id in dict_.keys():
    
    if not id == 'desconegut':
        # We divide by 10 because it's the number of images per class in the validation set.
        print id, dict_[id]/10

mercat_independencia 0.13933528359
societat_general 0.241627870337
farmacia_albinyana 0.377998333511
ajuntament 0.759791571174
mnactec 0.168620482365
escola_enginyeria 0.206969113069
masia_freixa 0.0951695608574
castell_cartoixa 0.182445905157
dona_treballadora 0.0866802743736
catedral 0.191894538954
teatre_principal 0.366097426245
estacio_nord 0.105029764362


In [9]:
query_id = '11962-11431-11239'
ER.single_eval(params,query_id)

escola_enginyeria
450
Displaying...


#### Hem aconseguit un MAP d'uns 0'24, encara que va variant cada vegada que s'executa. I no hem aconseguit que fes Display dins del jupyter, s'obre una finestra.